In [ ]:
from flask import Flask, render_template, request, jsonify
from transformers import pipeline
import os
from transformers.utils import is_flash_attn_2_available

print("\nCUTLASS_PATH :", os.getenv("CUTLASS_PATH"))
print('Flash attention enabled : ',is_flash_attn_2_available())
# Initialize the Flask application
app = Flask(__name__)

# Initialize the conversational pipeline
chatbot = pipeline('text-generation', model='ilsp/Meltemi-7B-Instruct-v1.5', device="cuda", flash_attention_2=True)
# pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1", trust_remote_code=True, device="cpu")
# pipe = pipeline("text-generation", model="deepseek-ai/deepseek-coder-33b-instruct", trust_remote_code=True, device="cpu")



/home/ekats/scratch/.env/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4cf42f5210>>
Traceback (most recent call last):
  File "/home/ekats/scratch/.env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 



CUTLASS_PATH : None
Flash attention enabled :  True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
model_name = 'ilsp/Meltemi-7B-Instruct-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model_pipeline = pipeline("text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        flash_attention_2=True, 
                        device=0 if torch.cuda.is_available() else -1)

/home/ekats/scratch/.env/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [1]:
from huggingface_hub import HfApi

huggingface_token = "your_huggingface_token"

api = HfApi()
files = api.list_repo_tree(
    repo_id="ilsp/Meltemi-7B-Instruct-v1.5", 
    revision="main", 
    recursive=True, 
    token=huggingface_token  # Ensure token is passed
)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "ilsp/Meltemi-7B-Instruct-v1.5"
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,  trust_remote_code=True)

# Check if flash_attention_2 is supported
model_config = model.config.to_dict()
supports_flash_attention = "flash_attention_2" in model_config
print(supports_flash_attention)

# Create pipeline with optional flash_attention_2
pipeline_kwargs = {"model": model, "tokenizer": tokenizer}
if supports_flash_attention:
    pipeline_kwargs["flash_attention_2"] = True

# Set device
pipeline_kwargs["device"] = 0 if torch.cuda.is_available() else -1

# Initialize pipeline
model_pipeline = pipeline("text-generation", **pipeline_kwargs)

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/48.0k [00:00<?, ?B/s]

model-00001-of-000004.safetensors:   0%|          | 0.00/8.71G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


False


OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 44.35 GiB of which 97.00 MiB is free. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 43.65 GiB is allocated by PyTorch, and 1.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlShutdown

def get_free_vram():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)  # GPU index 0
    info = nvmlDeviceGetMemoryInfo(handle)
    nvmlShutdown()
    return info.free / (1024 ** 3)  # Convert bytes to GB

# URL of the website you want to scrape
# model_name = 'deepseek-ai/deepseek-coder-33b-instruct'
model_name ='ilsp/Meltemi-7B-Instruct-v1.5'
# model_name = "deepseek-ai/DeepSeek-R1"
url = f'https://huggingface.co/{model_name}/tree/main'

# Send a GET request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the links on the webpage
    links = soup.find_all('a')
    
    safetensors = []
    for link in links:
        href = link.get('href')
        if href and f'.safetensors?download=true' in href:
            safetensors.append(href)
            print(href)
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')

free_vram = np.around(get_free_vram(),decimals=0)
model_ram = len(safetensors)*10 if len(safetensors)<5 else len(safetensors)*15
print(free_vram, model_ram)

In [29]:
from huggingface_hub import HfApi

def get_model_size(repo_id, branch="main"):
    api = HfApi()
    files = api.list_repo_tree(repo_id=repo_id, revision=branch, recursive=True)

    # Sum sizes only for files (not folders)
    # print([file.path for file in files if hasattr(file,"size") and ".safetensors" in file.path ])
    total_size = sum(file.size for file in files if hasattr(file, "size") and ".safetensors" in file.path )
    
    return total_size / (1024**3)  # Convert bytes to GB

# repo_id = "ilsp/Meltemi-7B-Instruct-v1"  # Change to your model
repo_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  # Change to your model

model_size_gb = get_model_size(repo_id)

print(f"Estimated model size: {model_size_gb} GB")


Estimated model size: 14.957581268623471 GB


In [30]:
# (((38056-9510)/1024)-13.937195261)/13.937195261 
# (((38060-9510)/1024)-13.937195261)/13.937195261
(((40144-9510)/1024)-14.957581268)/14.957581268

1.0000570338870112